Cassandra Python Assignment

In [27]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [28]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [29]:
session.execute("DROP KEYSPACE IF EXISTS m14python")

In [30]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [31]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14python WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [32]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
m14python
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a inventory table for testing

In [33]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14python.warehouse( 
    SKU INT PRIMARY KEY, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT
    );
""")


### Load data

In [34]:
df = pd.read_csv('data1.csv')
df

,sku,name,description,warehouse_num
0,1000,'iPhone','Apple',1
1,1001,'Galaxy S21','Samsung',2
2,1002,'Pixel 5','Google',3
3,1003,'OnePlus 9 Pro','OnePlus',4
4,1004,'Xiaomi Mi 11','Xiaomi',1


In [35]:
for index, row in df.iterrows():
    print(f"sku = {row.sku}, name ={row.name}, description = {row.description},  warehouse_num = {row.warehouse_num}")

sku = 1000, name =0, description = 'Apple',  warehouse_num = 1
sku = 1001, name =1, description = 'Samsung',  warehouse_num = 2
sku = 1002, name =2, description = 'Google',  warehouse_num = 3
sku = 1003, name =3, description = 'OnePlus',  warehouse_num = 4
sku = 1004, name =4, description = 'Xiaomi',  warehouse_num = 1


In [36]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES ({row[0]},{row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES (1000, 'iPhone', 'Apple', 1);
        

        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES (1001, 'Galaxy S21', 'Samsung', 2);
        

        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES (1002, 'Pixel 5', 'Google', 3);
        

        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES (1003, 'OnePlus 9 Pro', 'OnePlus', 4);
        

        INSERT INTO m14python.warehouse (sku,name,description, warehouse_num)     
        VALUES (1004, 'Xiaomi Mi 11', 'Xiaomi', 1);
        


In [37]:
rows = session.execute("select (sku,name,description,warehouse_num) from m14python.warehouse")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=1004, name=Xiaomi Mi 11, description=Xiaomi, warehouse_num=1
sku=1001, name=Galaxy S21, description=Samsung, warehouse_num=2
sku=1003, name=OnePlus 9 Pro, description=OnePlus, warehouse_num=4
sku=1002, name=Pixel 5, description=Google, warehouse_num=3
sku=1000, name=iPhone, description=Apple, warehouse_num=1


In [38]:
session.execute("CREATE INDEX IF NOT EXISTS warehouse_index ON m14python.warehouse(warehouse_num)")

In [39]:
result = session.execute("select (sku,description,name, warehouse_num) from m14python.warehouse where warehouse_num = 01")
for row in result:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, Warehouse_num={row[0][3]}")

1004, Xiaomi, Xiaomi Mi 11, Warehouse_num=1
1000, Apple, iPhone, Warehouse_num=1
